# Paths and Imports

In [1]:
import numpy as np
import pandas as pd
import plotnine
from plotnine import *
import pydicom

plotnine.options.figure_size = (12, 8)

In [3]:
PATH = 'E:/MyDocuments/GitHub/curriculum/dig_trans_healthcare/Course/data_quality/'

In [39]:
tex_theme = theme(plot_background = element_rect(fill = '#fafafa', color = '#fafafa'), axis_text_x=element_text(size=12),
         axis_text_y=element_text(size=12), axis_title_x=element_text(size=15), axis_title_y=element_text(size=15),
        legend_background = element_rect(fill="#fafafa"), legend_title=element_text(size=15),legend_text=element_text(size=12),
         legend_position=(.8,.7))

# Temperature Data

In [41]:
df_temp = pd.read_csv(f'{PATH}data/temp_value_counts.csv')
df_temp.head()

,COUNT(FINDINGID),NUMERICFINDING
0,3452,NaN
1,1,998898.0
2,1,888898.1
3,1,98245.9
4,1,98177.0


In [42]:
df_temp.drop(index=0, inplace=True)
df_temp.NUMERICFINDING = df_temp.NUMERICFINDING.astype(int)
df_temp = df_temp.groupby('NUMERICFINDING', as_index=False).sum()
df_temp['log_find'] = np.log(df_temp['COUNT(FINDINGID)'])

In [45]:
p = (ggplot(df_temp, aes(x='NUMERICFINDING', y='COUNT(FINDINGID)')) 
     + geom_bar(stat='identity') 
     + scale_y_log10() 
     + coord_cartesian(xlim=(0,250))
     + xlab('Temperature (F)')
     + ylab('Log Count')
     + tex_theme
    )
# p 
p.save(f'{PATH}images/temp_value_counts.pdf')

E:\Anaconda3\envs\curriculum\lib\site-packages\plotnine\ggplot.py:708: UserWarning: Saving 12 x 8 in image.
  from_inches(height, units), units))
E:\Anaconda3\envs\curriculum\lib\site-packages\plotnine\ggplot.py:709: UserWarning: Filename: E:/MyDocuments/GitHub/curriculum/dig_trans_healthcare/Course/data_quality/images/temp_value_counts.pdf
  warn('Filename: {}'.format(filename))


In [49]:
df_temp2 = pd.read_csv(f'{PATH}data/pt_temp_values.csv')
df_temp2.FINDINGDATETIME = pd.to_datetime(df_temp2.FINDINGDATETIME, format='%Y-%m-%d %H:%M:%S')
df_temp2.head()

,FINDINGDATETIME,NUMERICFINDING
0,2005-05-19 20:17:12,97.0
1,2005-05-20 00:37:58,98.9
2,2005-05-20 08:30:06,98.9
3,2012-05-26 03:50:27,97.8
4,2012-05-26 06:03:50,97.7


In [76]:
p = (ggplot(df_temp2.loc[(df_temp2.FINDINGDATETIME>'2012-12-01') & (df_temp2.FINDINGDATETIME<'2016-01-01'),:], aes('FINDINGDATETIME','NUMERICFINDING')) 
     + geom_point(size=3, color='b')
     + scale_x_date(date_breaks = '1 year', date_labels = '%Y')
     + tex_theme
#      + theme(axis_text_x = element_text(angle=90, hjust=1))
    )
p
p.save(f'{PATH}images/pt_temp_values.pdf', verbose=False)

In [85]:
df_ass = pd.read_csv(f'{PATH}data/pt_temp_associated.csv')
df_ass.FINDINGDATETIME = pd.to_datetime(df_ass.FINDINGDATETIME, format='%Y-%m-%d %H:%M:%S')
df_ass.FINDINGDATETIME = df_ass.FINDINGDATETIME.dt.date
df_ass.dropna(inplace=True)
df_ass

,FINDINGDATETIME,FINDINGDESC,NUMERICFINDING
0,2014-12-26,PULSE OXIMETRY,97.00
1,2014-12-26,WEIGHT/SCALE (ounces),2800.16
2,2014-12-26,HEIGHT (inches),62.00
4,2014-12-26,Diastolic Blood Pressure,82.00
5,2014-12-26,Systolic Blood Pressure,139.00
7,2014-12-26,HEIGHT (CM),157.48
8,2014-12-26,PULSE,75.00
9,2014-12-26,BODY MASS INDEX,32.13
11,2014-12-26,O2 SAT%,97.00
12,2014-12-26,TEMPERATURE (F),0.00


In [87]:
print(df_ass.to_latex(index=False))

\begin{tabular}{llr}
\toprule
FINDINGDATETIME &               FINDINGDESC &  NUMERICFINDING \\
\midrule
     2014-12-26 &            PULSE OXIMETRY &           97.00 \\
     2014-12-26 &     WEIGHT/SCALE (ounces) &         2800.16 \\
     2014-12-26 &           HEIGHT (inches) &           62.00 \\
     2014-12-26 &  Diastolic Blood Pressure &           82.00 \\
     2014-12-26 &   Systolic Blood Pressure &          139.00 \\
     2014-12-26 &               HEIGHT (CM) &          157.48 \\
     2014-12-26 &                     PULSE &           75.00 \\
     2014-12-26 &           BODY MASS INDEX &           32.13 \\
     2014-12-26 &                   O2 SAT\% &           97.00 \\
     2014-12-26 &           TEMPERATURE (F) &            0.00 \\
     2014-12-26 &   Systolic Blood Pressure &          139.00 \\
     2014-12-26 &               WEIGHT (KG) &           79.38 \\
     2014-12-26 &  Diastolic Blood Pressure &           82.00 \\
\bottomrule
\end{tabular}



# DICOM

In [18]:
ds = pydicom.dcmread(f'{PATH}data/356248481.dcm', stop_before_pixels=True)

In [ ]:
ds.

In [14]:
(ds.data_element('LastMenstrualDate').VR)

'DA'

In [ ]:
ds.

In [26]:
x.name


'Group Length'

In [30]:
name_list = []
type_list = []
val_list = []
for x in ds.iterall():
    name_list.append(x.name)
    type_list.append(x.VR)
    val_list.append(x.value)
df = pd.DataFrame({'name':name_list,'VR':type_list,'value':val_list})

In [38]:
print(df.loc[:30,:].to_latex(index=False))

\begin{tabular}{lll}
\toprule
                        name &  VR &                                              value \\
\midrule
                Group Length &  UL &                                                532 \\
                  Image Type &  CS &                                            DERIVED \\
               SOP Class UID &  UI &                          1.2.840.10008.5.1.4.1.1.2 \\
            SOP Instance UID &  UI &          1.2.840.114356.2008.11.30.12.34.2.329.999 \\
                  Study Date &  DA &                                           20081230 \\
                Content Date &  DA &                                           20081230 \\
                  Study Time &  TM &                                             122731 \\
                Content Time &  TM &                                         12299.0000 \\
            Accession Number &  SH &                                            7421249 \\
                    Modality &  CS &               

In [19]:
[x for x in ds.iterall()]

[(0008, 0000) Group Length                        UL: 532,
 (0008, 0008) Image Type                          CS: 'DERIVED',
 (0008, 0016) SOP Class UID                       UI: CT Image Storage,
 (0008, 0018) SOP Instance UID                    UI: 1.2.840.114356.2008.11.30.12.34.2.329.999,
 (0008, 0020) Study Date                          DA: '20081230',
 (0008, 0023) Content Date                        DA: '20081230',
 (0008, 0030) Study Time                          TM: '122731',
 (0008, 0033) Content Time                        TM: '12299.0000',
 (0008, 0050) Accession Number                    SH: '7421249',
 (0008, 0060) Modality                            CS: 'CT',
 (0008, 0080) Institution Name                    LO: 'Manhasset Diagnostic Imaging',
 (0008, 0090) Referring Physician's Name          PN: 'HOUTEN,JOHN K',
 (0008, 1010) Station Name                        SH: '',
 (0008, 1030) Study Description                   LO: 'MOSES CT Outside Reference Images',
 <Sequence, 

# Next